<a href="https://colab.research.google.com/github/sgogoigh/Fake-News-Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset - https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets

In [53]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [56]:
pd1 = pd.read_csv('Fake.csv', on_bad_lines='skip')
pd2 = pd.read_csv('True.csv')

In [57]:
pd1.shape

(23481, 4)

In [58]:
pd2.shape

(21417, 4)

In [59]:
pd1['label'] = 1
pd2['label'] = 0

In [60]:
pd1.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [61]:
pd2.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [62]:
# concatenating and then shuffling rows
df = pd.concat([pd1, pd2], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

In [63]:
df.head()

,title,text,subject,date,label
0,Trump close to naming congressman Price as hea...,(Reuters) - U.S. President-elect Donald Trump ...,politicsNews,"November 29, 2016",0
1,Factbox: Trump on Twitter (Aug 1) - Stock mark...,The following statements were posted to the ve...,politicsNews,"August 1, 2017",0
2,Leaked G20 Video Appears To Show Trump Rubbin...,The first official meeting between Trump and R...,News,"July 7, 2017",1
3,First lady Melania plays tourist as Trump depa...,BEIJING (Reuters) - As U.S. President Donald T...,politicsNews,"November 10, 2017",0
4,MIZZOU Cry-Babies Complain Paris Terror Traged...,Never mind that almost every single aspect of ...,left-news,"Nov 14, 2015",1


In [64]:
# no null values
df.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [65]:
# there seem to be duplicates
df.duplicated().sum()

np.int64(209)

In [66]:
df.shape

(44898, 5)

In [67]:
df = df.drop_duplicates()

In [68]:
df.shape

(44689, 5)

In [72]:
X = df.drop(columns=['label','date'], axis=1)
Y = df['label']

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44689 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44689 non-null  object
 1   text     44689 non-null  object
 2   subject  44689 non-null  object
 3   date     44689 non-null  object
 4   label    44689 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.0+ MB


## Stemming

In [73]:
port_stem = PorterStemmer()

In [74]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [78]:
df['content'] = df['title'] + ' ' + df['subject']

In [79]:
df['content'] = df['content'].apply(stemming)

In [80]:
X = df['content'].values
Y = df['label'].values

## Vector Representation

In [81]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

## Training & Testing

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [84]:
# training data accuracy
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
training_data_accuracy

0.9999160862633213

In [85]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
test_data_accuracy

0.9998881181472365